In [1]:
import rospy
from std_msgs.msg import String
from geometry_msgs.msg import Twist
from sensor_msgs.msg import LaserScan
from move_base_msgs.msg import MoveBaseAction, MoveBaseGoal, MoveBaseActionGoal, MoveBaseActionFeedback
from actionlib_msgs.msg import GoalID, GoalStatusArray
from nav_msgs.msg import Odometry
from tf.transformations import *

import jupyros as jr
from jupyros import ros3d
import ipywidgets as widgets
from ipywidgets import interact, interactive, fixed, interact_manual, Layout
from IPython.display import display, clear_output

import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
import numpy as np
import math
import time
import os

import getch
import sys, tty
sys.path.append("../src/")
import option1
import option2
import option3

In [2]:
# publishers
rospy.init_node('jupyter_node')
publisher_movebase = jr.publish("move_base/goal", MoveBaseActionGoal)
publisher_cancel = jr.publish("move_base/cancel", GoalID)
publisher_velocity2 = jr.publish("cmd_vel", Twist)
publisher_velocity3 = jr.publish("cmd_vel", Twist)

In [3]:
### Odometry
x_odom, y_odom = [], [] 

### Status
colors, goal_list, feedback_list = ["white"], [0], [0] 

def odometry_status(position):
    global x_odom
    global y_odom   
    x_odom.append(position.pose.pose.position.x)
    y_odom.append(position.pose.pose.position.y)

def robot_status(status):
    # This function updates the global variables used to print the bar plot when a new message arrives.
    global colors
    adder = sum(feedback_list)   # Sum of feedbackcs until the last call of this function.
    
    if status.status_list != []:    # If a goal was processed.
        n = status.status_list[0].status   # Status of the goal.
    
        if len(colors) < ad.goal_counter + 1:   # If a new goal was sent, it increases the length of arrays.
            colors.append("blue")
            goal_list.append(ad.goal_counter)
            feedback_list.append(ad.feedback_counter - adder)
            
        else:   # It updates the value of the last goal's feedbacks.
            feedback_list[-1] = feedback_list[-1] + ad.feedback_counter - adder
        
        # It sets the color of the last bar depending on the goal status.
        if n == 1:
            colors[-1] = "blue"
        
        if n == 3:
            colors[-1] = "green"
            
        if n in [2, 4, 5, 8]:
            colors[-1] = "red"

# subscribers
sub_status = rospy.Subscriber('/move_base/status', GoalStatusArray, status_callback)
sub_odom = rospy.Subscriber('/odom', Odometry, odom_callback)

NameError: name 'status_callback' is not defined

In [4]:
v = ros3d.Viewer()
rc = ros3d.ROSConnection(url="ws://localhost:9090") 
tf_client = ros3d.TFClient(ros=rc, fixed_frame='map')
laser_view = ros3d.LaserScan(topic="/scan", ros=rc, tf_client=tf_client) 
map_view = ros3d.OccupancyGrid(topic="/map", ros=rc, tf_client=tf_client, continuous=True) 
path = ros3d.Path(topic="/move_base/NavfnROS/plan", ros=rc, tf_client=tf_client) 
urdf = ros3d.URDFModel(ros=rc, tf_client=tf_client, path=os.environ.get('JUPYROS_ASSETS_URL', 'http://localhost:3000')) 
g = ros3d.GridModel()
v.objects = [g, laser_view, map_view, path, urdf]
v

Viewer(objects=[GridModel(), LaserScan(ros=ROSConnection(url='ws://localhost:9090'), tf_client=TFClient(fixed_…

In [ ]:
########### MAIN ############

# User interface to give the user a set of options
def welcome():
    """
    This is the instruction list for user
    """
    
    print("Whats Up? This is RT2 Assignment, and here is a Jupyter Notebook version for it.")
    print("Press [1] for autonomous drive of the robot")
    print("Press [2] for manual drive of the robot")
    print("Press [3] for manual drive of the robot with collision avoidance")
    print("Press [4] for exitting from the system")


def main():

    """
    This function used to choose the driving mode by user
    #The user message is passed to the service ``master``
    """

    exit_system = False
    rospy.init_node('jupyter_node')
    while not rospy.is_shutdown() and not exit_system:
        welcome()
        while True:
            try:
                button = int(input("Choose drive options: "))
                break
            except:
                print("Your input is not possible to obtain")
        if button == 1:
            while not option1.option_one():
                pass
        if button == 2:
            while not option2.option_two():
                pass
        if button == 3:
            while not option3.option_three():
                pass
        elif button == 4:
            exit_system = True
        else:
            print("Wrong button")

if __name__ == '__main__':
    main()

Whats Up? This is RT2 Assignment, and here is a Jupyter Notebook version for it.
Press [1] for autonomous drive of the robot
Press [2] for manual drive of the robot
Press [3] for manual drive of the robot with collision avoidance
Press [4] for exitting from the system
Choose drive options: 2
Manual drive is activated
Press [W] or [w]: to move forward
Press [S] or [s]: to move backward
Press [A] or [a]: to turn left
Press [D] or [d]: to turn right
Press [Q] or [q]: to exit from manual drive 

Press: w

Press: w

Press: s


In [ ]:
%matplotlib notebook

laser_rmax = 11   # Max radius of the laser scanner plot.

# Creates figure
fig = plt.figure(figsize=(9,10))

# Creates axes.
ax1 = plt.subplot(2, 2, (1, 2), projection='polar')   
ax2 = plt.subplot(2, 2, 3)
ax3 = plt.subplot(2, 2, 4)
        
# Creates line objects for the plots
laser_ln, = ax1.plot([], [], 'r.')      
odom_ln, = ax2.plot([], [], 'ro')

def update_plot(frame):
    # This function, called by funcAnimation, updates the plots to animate them.
    
    # Set the new data of the laser scanner.
    laser_ln.set_xdata(laser_theta)
    laser_ln.set_ydata(laser_r)
    ax1.set_rmax(laser_rmax)   # Resize the max radius of the laser scanner plot.
    
    # Set the new data of the odometry.
    odom_ln.set_data(odom_x, odom_y)
    
    # Set the new data of the goal history.
    ax3.bar(goal_list[1::], feedback_list[1::], width=0.8, color=colors[1::])
    if feedback_list[-1] > 500:   # If the number of feedbacks is out of range, resize the plot y limit.
        ax3.set_ylim(0, feedback_list[-1] + 10)
    
def plot_init():
    # This function, called by funcAnimation, initilizes the plots.
    
    # Initialization of the laser scanner plot.
    ax1.set_title("Laser Scanner", fontsize=20)
    ax1.set_xlabel('[m]', fontsize=10)
    ax1.set_thetalim(-np.pi/2, np.pi/2)
    ax1.set_rmax(laser_rmax)
    ax1.set_theta_zero_location("N")
    
    # Initialization of the odometry plot.
    ax2.set_title("Odometry trajectory", fontsize=20)
    ax2.set_xlabel('x [m]', fontsize=10)
    ax2.set_ylabel('y [m]', fontsize=10)
    ax2.set_xlim(-10, 10) 
    ax2.set_ylim(-10, 10)
    ax2.grid()
    
    # Initialization of the goal history plot.
    ax3.set_title("Goal history", fontsize=20)
    
    ax3.set_xlabel('goal', fontsize=10)
    ax3.set_ylabel('feedbacks', fontsize=10)
    ax3.set_ylim(0, 500)
    ax3.grid(axis='y')
    ax3.xaxis.set_major_locator(MaxNLocator(integer=True))
    
    # Legend for the goal history plot.
    legend = {'goal reached':'green', 
              'goal canceled':'red', 
              'goal in progress':'blue'}         
    labels = list(legend.keys())
    handles = [plt.Rectangle((0,0),1,1, color=legend[label]) for label in labels]
    ax3.legend(handles, labels)

ani = FuncAnimation(fig, update_plot, init_func=plot_init, interval=500, blit=True)
plt.show()